# 04_03: Using Knowledge Graphs in a GraphRAG Pipeline

In [ ]:
URI = "..."
USER = "neo4j"
PWD = "..."
OPENAI_API_KEY = "..."

In [ ]:
from langchain_neo4j import Neo4jGraph
from langchain_core.documents import Document
from langchain_community.graphs.graph_document import GraphDocument
import os

from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_neo4j import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
graph = Neo4jGraph(url=URI, username=USER, password=PWD)
print(graph.schema)

In [ ]:
enhanced_graph = Neo4jGraph(url=URI, username=USER, password=PWD, enhanced_schema=True)
print(enhanced_graph.schema)

In [ ]:
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o", temperature=0)
chain = GraphCypherQAChain.from_llm(
    graph=enhanced_graph, llm=llm, verbose=True, allow_dangerous_requests=True
)
response = chain.invoke({"query": "What is slackcountry?"})
response

In [ ]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to
answer questions about skiing.
Convert the user's question based on the schema.
When you are presented with query properties such as id's like "alpine touring",
be sure to convert the first letter to capital case, such as "Alpine Touring"
before you run the Cypher query.

For example, if I were to ask "Tell me about telemark skiing," you should create
a Cypher query that finds all nodes with the id "Telemark Skiing" and then find
all nodes connected to those nodes and use those to forumulate your answer, like this:

MATCH (a {{id: "Telemark Skiing"}})-[r]-(b)
RETURN a, r, b

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=enhanced_graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
cypher_chain.invoke({"query": "Tell me about ski touring?"})